In [ ]:
import os
import sys

ENDWITHS = 'OpenMantra'

NOTEBOOK_DIR = os.getcwd()

if not NOTEBOOK_DIR.endswith(ENDWITHS):
    raise ValueError(f"Not in correct dir, expect end with {ENDWITHS}, but got {NOTEBOOK_DIR} instead")

BASE_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, '..', '..', '..', '..', '..'))
print(BASE_DIR)

sys.path.insert(0, os.path.join(BASE_DIR, 'code'))

import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

In [ ]:
from OpenMantraEvaluator import OpenMantraEvaluator
from pipeline.TranslationModels.LLMPerImageTranslator import LLMPerImageTranslator

In [ ]:
from dotenv import load_dotenv
load_dotenv(os.path.abspath(os.path.join(BASE_DIR, '..', '.env')))

HF_TOKEN = os.getenv('HF_TOKEN')

In [ ]:
OPENMANTRA_ROOT = os.path.join(BASE_DIR, 'data', 'open-mantra-dataset')
print(f"OpenMantra dataset root: {OPENMANTRA_ROOT}")

In [ ]:
MODEL_NAME = "Qwen/Qwen2.5-0.5B"

evaluator = OpenMantraEvaluator(
    openmantra_root=OPENMANTRA_ROOT,
    source_lang='text_ja',
    target_lang='text_en'
)

translation_model = LLMPerImageTranslator()
translation_model.load_model(device=device, model_name=MODEL_NAME)

# Configure predict parameters
translation_model.configure(
    max_new_tokens=30,
    temperature=0.3,
    use_batch=True
)

In [ ]:
metrics = evaluator.evaluate(translation_model, device=device, verbose=True, model_name=MODEL_NAME)

In [ ]:
translation_model.unload_model()

In [ ]:
for key, value in metrics.items():
    print(f"  {key}: {value:.4f}")